# Colab Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/study/Github/MLDL1/HW3

/content/drive/MyDrive/study/Github/MLDL1/HW3


# Import Modules

In [ ]:
import copy
import numpy as np
import matplotlib.pyplot as plt
from mnist.data_utils import load_data

#Utils

In [ ]:
def sigmoid(z):
    """
    Do NOT modify this function
    """
    return 1/(1+np.exp(-z))

def softmax(X):
    """
    Do NOT modify this function
    """
    logit = np.exp(X-np.amax(X, axis=1, keepdims=True))
    numer = logit
    denom = np.sum(logit, axis=1, keepdims=True)
    return numer/denom

def relu(X):
    return np.where(X>0,X,0)
def load_batch(X, Y, batch_size, shuffle=True):
    """
    Generates batches with the remainder dropped.

    Do NOT modify this function
    """
    if shuffle:
        permutation = np.random.permutation(X.shape[0])
        X = X[permutation, :]
        Y = Y[permutation, :]
    num_steps = int(X.shape[0])//batch_size
    step = 0
    while step<num_steps:
        X_batch = X[batch_size*step:batch_size*(step+1)]
        Y_batch = Y[batch_size*step:batch_size*(step+1)]
        step+=1
        yield X_batch, Y_batch

#2-Layer Neural Network

In [41]:
class TwoLayerNN:
    """ a neural network with 2 layers """

    def __init__(self, input_dim, num_hiddens, num_classes):
        """
        Do NOT modify this function.
        """
        self.input_dim = input_dim
        self.num_hiddens = num_hiddens
        self.num_classes = num_classes
        self.params = self.initialize_parameters(input_dim, num_hiddens, num_classes)

    def initialize_parameters(self, input_dim, num_hiddens, num_classes):
        """
        initializes parameters with Xavier Initialization.

        Question (a)
        - refer to https://paperswithcode.com/method/xavier-initialization for Xavier initialization 
        
        Inputs
        - input_dim
        - num_hiddens
        - num_classes
        Returns
        - params: a dictionary with the initialized parameters.
        """

        params = {}
        bound = 1/np.sqrt(input_dim)
        params["W1"] = np.random.uniform(-bound, bound, size=(input_dim,num_hiddens))
        params["b1"] = np.zeros((num_hiddens,))
        params["W2"] = np.random.uniform(-bound, bound, size=(num_hiddens,num_classes))
        params["b2"] = np.zeros((num_classes,))

        return params

    def forward(self, X):
        """
        Define and perform the feed forward step of a two-layer neural network.
        Specifically, the network structure is given by

          y = softmax(sigmoid(X W1 + b1) W2 + b2)

        where X is the input matrix of shape (N, D), y is the class distribution matrix
        of shape (N, C), N is the number of examples (either the entire dataset or
        a mini-batch), D is the feature dimensionality, and C is the number of classes.

        Question (b)
        - ff_dict will be used to run backpropagation in backward method.

        Inputs
        - X: the input matrix of shape (N, D)

        Returns
        - y: the output of the model
        - ff_dict: a dictionary with all the fully connected units and activations.
        """

        ff_dict = {}
        ff_dict["s1"] = sigmoid(X.dot(self.params["W1"]) + self.params["b1"])
        ff_dict["s2"] = softmax(ff_dict["s1"].dot(self.params["W2"]) + self.params["b2"])
        y = ff_dict["s2"]
        return y, ff_dict

    def backward(self, X, Y, ff_dict):
        """
        Performs backpropagation over the two-layer neural network, and returns
        a dictionary of gradients of all model parameters.

        Question (c)

        Inputs:
         - X: the input matrix of shape (B, D), where B is the number of examples
              in a mini-batch, D is the feature dimensionality.
         - Y: the matrix of one-hot encoded ground truth classes of shape (B, C),
              where B is the number of examples in a mini-batch, C is the number
              of classes.
         - ff_dict: the dictionary containing all the fully connected units and
              activations.

        Returns:
         - grads: a dictionary containing the gradients of corresponding weights and biases.
        """
        grads = {}
        # loss function: cross entropy
        grads_softmax = ff_dict["s2"] - np.where(Y == 1,1,0)
        grads["db2"] = grads_softmax.sum(axis=0)
        grads["dW2"] = ff_dict["s1"].T.dot(grads_softmax)

        grads_s1 = grads_softmax.dot(self.params["W2"].T)
        grads_sigmoid = grads_s1*(ff_dict["s1"]*(1-ff_dict["s1"]))
        grads["db1"] = grads_sigmoid.sum(axis=0)
        grads["dW1"] = X.T.dot(grads_sigmoid)

        return grads

    def compute_loss(self, Y, Y_hat):
        """
        Computes cross entropy loss.

        Do NOT modify this function.

        Inputs
            Y:
            Y_hat:
        Returns
            loss:
        """
        loss = -(1/Y.shape[0]) * np.sum(np.multiply(Y, np.log(Y_hat)))
        return loss

    def train(self, X, Y, X_val, Y_val, lr, n_epochs, batch_size, log_interval=1):
        """
        Runs mini-batch gradient descent.

        Do NOT Modify this method.

        Inputs
        - X
        - Y
        - X_val
        - Y_Val
        - lr
        - n_epochs
        - batch_size
        - log_interval
        """
        for epoch in range(n_epochs):
            for X_batch, Y_batch in load_batch(X, Y, batch_size):
                self.train_step(X_batch, Y_batch, batch_size, lr)
            if epoch % log_interval==0:
                Y_hat, ff_dict = self.forward(X)
                train_loss = self.compute_loss(Y, Y_hat)
                train_acc = self.evaluate(Y, Y_hat)
                Y_hat, ff_dict = self.forward(X_val)
                valid_loss = self.compute_loss(Y_val, Y_hat)
                valid_acc = self.evaluate(Y_val, Y_hat)
                print('epoch {:02} - train loss/acc: {:.3f} {:.3f}, valid loss/acc: {:.3f} {:.3f}'.\
                      format(epoch, train_loss, train_acc, valid_loss, valid_acc))

    def train_step(self, X_batch, Y_batch, batch_size, lr):
        """
        Updates the parameters using gradient descent.

        Do NOT Modify this method.

        Inputs
        - X_batch
        - Y_batch
        - batch_size
        - lr
        """
        _, ff_dict = self.forward(X_batch)
        grads = self.backward(X_batch, Y_batch, ff_dict)
        self.params["W1"] -= lr * grads["dW1"]/batch_size
        self.params["b1"] -= lr * grads["db1"]/batch_size
        self.params["W2"] -= lr * grads["dW2"]/batch_size
        self.params["b2"] -= lr * grads["db2"]/batch_size

    def evaluate(self, Y, Y_hat):
        """
        Computes classification accuracy.
        
        Do NOT modify this function

        Inputs
        - Y: A numpy array of shape (N, C) containing the softmax outputs,
             where C is the number of classes.
        - Y_hat: A numpy array of shape (N, C) containing the one-hot encoded labels,
             where C is the number of classes.

        Returns
            accuracy: the classification accuracy in float
        """        
        classes_pred = np.argmax(Y_hat, axis=1)
        classes_gt = np.argmax(Y, axis=1)
        accuracy = float(np.sum(classes_pred==classes_gt)) / Y.shape[0]
        return accuracy

#Load MNIST

In [ ]:
X_train, Y_train, X_test, Y_test = load_data()

idxs = np.arange(len(X_train))
np.random.shuffle(idxs)
split_idx = int(np.ceil(len(idxs)*0.8))
X_valid, Y_valid = X_train[idxs[split_idx:]], Y_train[idxs[split_idx:]]
X_train, Y_train = X_train[idxs[:split_idx]], Y_train[idxs[:split_idx]]
print()
print('Set validation data aside')
print('Training data shape: ', X_train.shape)
print('Training labels shape: ', Y_train.shape)
print('Validation data shape: ', X_valid.shape)
print('Validation labels shape: ', Y_valid.shape)

MNIST data loaded:
Training data shape: (60000, 784)
Training labels shape: (60000, 10)
Test data shape: (10000, 784)
Test labels shape: (10000, 10)

Set validation data aside
Training data shape:  (48000, 784)
Training labels shape:  (48000, 10)
Validation data shape:  (12000, 784)
Validation labels shape:  (12000, 10)


#Training & Evaluation

In [ ]:
### 
# Question (d)
# Tune the hyperparameters with validation data, 
# and print the results by running the lines below.
###

In [ ]:
# train the model
lr_lst = np.random.uniform(1,3,size=5)
n_epochs_lst = [15, 20, 25]
batch_size_lst = [128, 256, 512]
num_hiddens_lst = [32, 64, 128]
combinations = [(lr, n_epochs, batch_size, num_hiddens) 
                  for lr in lr_lst for n_epochs in n_epochs_lst 
                    for batch_size in batch_size_lst for num_hiddens in num_hiddens_lst]
for i, (lr, n_epochs, batch_size, num_hiddens) in enumerate(combinations):
  print('Combination {:02} - lr: {:.3f}, n_epochs: {}, batch_size: {}, num_hiddens: {}'.\
                      format(i, lr, n_epochs, batch_size, num_hiddens))
  model = TwoLayerNN(input_dim=784, num_hiddens=num_hiddens, num_classes=10)
  model.train(X_train, Y_train, X_valid, Y_valid, lr, n_epochs, batch_size)


Combination 00 - lr: 2.189, n_epochs: 15, batch_size: 128, num_hiddens: 32
epoch 00 - train loss/acc: 0.218 0.936, valid loss/acc: 0.230 0.929
epoch 01 - train loss/acc: 0.158 0.955, valid loss/acc: 0.180 0.945
epoch 02 - train loss/acc: 0.130 0.962, valid loss/acc: 0.158 0.953
epoch 03 - train loss/acc: 0.115 0.967, valid loss/acc: 0.150 0.956
epoch 04 - train loss/acc: 0.100 0.971, valid loss/acc: 0.141 0.959
epoch 05 - train loss/acc: 0.092 0.974, valid loss/acc: 0.138 0.960
epoch 06 - train loss/acc: 0.087 0.974, valid loss/acc: 0.137 0.959
epoch 07 - train loss/acc: 0.081 0.975, valid loss/acc: 0.137 0.960
epoch 08 - train loss/acc: 0.075 0.977, valid loss/acc: 0.134 0.961
epoch 09 - train loss/acc: 0.068 0.980, valid loss/acc: 0.130 0.961
epoch 10 - train loss/acc: 0.062 0.982, valid loss/acc: 0.127 0.963
epoch 11 - train loss/acc: 0.057 0.983, valid loss/acc: 0.128 0.964
epoch 12 - train loss/acc: 0.055 0.985, valid loss/acc: 0.124 0.963
epoch 13 - train loss/acc: 0.051 0.986, v

In [55]:
# the best combination of hyperparameters from cross-validation
model = TwoLayerNN(input_dim=784, num_hiddens=128, num_classes=10)
lr, n_epochs, batch_size = 1.590, 25, 128
model.train(X_train, Y_train, X_valid, Y_valid, lr, n_epochs, batch_size)

epoch 00 - train loss/acc: 0.246 0.926, valid loss/acc: 0.259 0.921
epoch 01 - train loss/acc: 0.175 0.948, valid loss/acc: 0.189 0.944
epoch 02 - train loss/acc: 0.137 0.960, valid loss/acc: 0.156 0.955
epoch 03 - train loss/acc: 0.113 0.968, valid loss/acc: 0.138 0.958
epoch 04 - train loss/acc: 0.091 0.974, valid loss/acc: 0.120 0.964
epoch 05 - train loss/acc: 0.079 0.978, valid loss/acc: 0.114 0.966
epoch 06 - train loss/acc: 0.072 0.980, valid loss/acc: 0.110 0.966
epoch 07 - train loss/acc: 0.059 0.984, valid loss/acc: 0.100 0.970
epoch 08 - train loss/acc: 0.057 0.984, valid loss/acc: 0.102 0.969
epoch 09 - train loss/acc: 0.052 0.985, valid loss/acc: 0.099 0.970
epoch 10 - train loss/acc: 0.045 0.988, valid loss/acc: 0.094 0.972
epoch 11 - train loss/acc: 0.037 0.991, valid loss/acc: 0.089 0.972
epoch 12 - train loss/acc: 0.034 0.992, valid loss/acc: 0.089 0.973
epoch 13 - train loss/acc: 0.032 0.993, valid loss/acc: 0.089 0.973
epoch 14 - train loss/acc: 0.027 0.995, valid lo

In [56]:
# evalute the model on test data
Y_hat, _ = model.forward(X_test)
test_loss = model.compute_loss(Y_test, Y_hat)
test_acc = model.evaluate(Y_test, Y_hat)
print("Final test loss = {:.3f}, acc = {:.3f}".format(test_loss, test_acc))

Final test loss = 0.070, acc = 0.978


# Extra Credit (Optional)

In [ ]:
def initialize_parameters(self, input_dim, num_hiddens, num_classes):
    """
    initializes parameters with He Initialization.

    Question (e)
    - refer to https://paperswithcode.com/method/he-initialization for He initialization 
    
    Inputs
    - input_dim
    - num_hiddens
    - num_classes
    Returns
    - params: a dictionary with the initialized parameters.
    """
    params = {}
    mu, sigma1, sigma2 = 0, np.sqrt(2/input_dim), np.sqrt(2/num_hiddens)
    params["W1"] = np.random.normal(mu,sigma1,size=(input_dim,num_hiddens))
    params["b1"] = np.zeros((num_hiddens,))
    params["W2"] = np.random.normal(mu,sigma2,size=(num_hiddens,num_classes))
    params["b2"] = np.zeros((num_classes,))

    return params

def forward_relu(self, X):
    """
    Defines and performs the feed forward step of a two-layer neural network.
    Specifically, the network structue is given by

        y = softmax(relu(X W1 + b1) W2 + b2)

    where X is the input matrix of shape (N, D), y is the class distribution matrix
    of shape (N, C), N is the number of examples (either the entire dataset or
    a mini-batch), D is the feature dimensionality, and C is the number of classes.

    Question (e)

    Inputs
        X: the input matrix of shape (N, D)

    Returns
        y: the output of the model
        ff_dict: a dictionary containing all the fully connected units and activations.
    """
    ff_dict = {}
    ff_dict["s1"] = relu(X.dot(self.params["W1"])) + self.params["b1"]
    ff_dict["s2"] = softmax(ff_dict["s1"].dot(self.params["W2"])) + self.params["b2"]
    y = ff_dict["s2"]

    return y, ff_dict

def backward_relu(self, X, Y, ff_dict):
    """
    Performs backpropagation over the two-layer neural network, and returns
    a dictionary of gradients of all model parameters.

    Question (e)

    Inputs:
        - X: the input matrix of shape (B, D), where B is the number of examples
            in a mini-batch, D is the feature dimensionality.
        - Y: the matrix of one-hot encoded ground truth classes of shape (B, C),
            where B is the number of examples in a mini-batch, C is the number
            of classes.
        - ff_dict: the dictionary containing all the fully connected units and
            activations.

    Returns:
        - grads: a dictionary containing the gradients of corresponding weights
            and biases.
    """
    grads = {}
    grads_softmax = ff_dict["s2"] - np.where(Y==1,1,0)
    grads["W2"] = ff_dict["s1"].T.dot(grads_softmax)
    grads["b2"] = np.sum(grads_softmax, axis=0)

    grads_s1 = grads_softmax.dot(self.params["W2"].T)
    grads_relu = grads_s1 * np.where(ff_dict["s1"]>0,1,0)
    grads["W1"] = X.T.dot(grads_relu)
    grads["b1"] = np.sum(grads_relu, axis=0)

    return grads

TwoLayerNNRelu = copy.copy(TwoLayerNN)
TwoLayerNNRelu.initialize_parameters = initialize_parameters
TwoLayerNNRelu.feed_forward = forward_relu
TwoLayerNNRelu.back_propagate = backward_relu

In [ ]:
### 
# Question (e)
# Tune the hyperparameters with validation data,
# and print the results by running the lines below.
###

In [39]:
# model instantiation
model_relu = TwoLayerNNRelu(input_dim=784, num_hiddens=64, num_classes=10)

In [40]:
# train the model
lr, n_epochs, batch_size = 1.5, 20, 256
history = model_relu.train(X_train, Y_train, X_valid, Y_valid, lr, n_epochs, batch_size)

epoch 00 - train loss/acc: 0.289 0.917, valid loss/acc: 0.294 0.912
epoch 01 - train loss/acc: 0.230 0.934, valid loss/acc: 0.240 0.930
epoch 02 - train loss/acc: 0.191 0.945, valid loss/acc: 0.204 0.940
epoch 03 - train loss/acc: 0.172 0.950, valid loss/acc: 0.189 0.943
epoch 04 - train loss/acc: 0.145 0.958, valid loss/acc: 0.166 0.949
epoch 05 - train loss/acc: 0.131 0.963, valid loss/acc: 0.152 0.956
epoch 06 - train loss/acc: 0.119 0.967, valid loss/acc: 0.145 0.957
epoch 07 - train loss/acc: 0.106 0.971, valid loss/acc: 0.134 0.961
epoch 08 - train loss/acc: 0.100 0.972, valid loss/acc: 0.129 0.961
epoch 09 - train loss/acc: 0.090 0.974, valid loss/acc: 0.121 0.965
epoch 10 - train loss/acc: 0.083 0.977, valid loss/acc: 0.116 0.964
epoch 11 - train loss/acc: 0.078 0.978, valid loss/acc: 0.114 0.966
epoch 12 - train loss/acc: 0.072 0.980, valid loss/acc: 0.108 0.968
epoch 13 - train loss/acc: 0.069 0.981, valid loss/acc: 0.108 0.968
epoch 14 - train loss/acc: 0.063 0.983, valid lo

In [ ]:
Y_hat, _ = model_relu.forward(X_test)
test_loss = model_relu.compute_loss(Y_test, Y_hat)
test_acc = model_relu.evaluate(Y_test, Y_hat)
print("Final test loss = {:.3f}, acc = {:.3f}".format(test_loss, test_acc))